### pivoting for cuda LU   
 

In [ ]:
python run.py  --opt 1 --size 3 --pivoting 0  --check_equal 1 \
  --verbose false --mc_iter 100
python run.py  --opt 1 --size 3 --pivoting 1  --check_equal 1 \
  --verbose false --mc_iter 100

#### cuda without pivoting 
```        
2 2 8    k=0  val_check = 2      2   2 8
7 7 10   1st column/val_check    3.5 7 10      
6 8 1                            3   8 1      
                                                 2   2   8
  7 10  -=  3.5  @ 2  8  -= 7 28   = 0 -18       3.5 0 -18
  8 1         3             6 24     2 -23       3   2 -23

2nd round :  val_check = 0   we cant /val_check

2nd column /0???
```

#### cuda pivoting  
```
P = 0,1,2                          P = 1,1,2
2 2 8   k=0 idx1based-1=1(val7)   7 7 10
7 7 10  pivotRow = 1              2 2 8 
6 8 1    rowswap(row1,row0=rowk)  6 8 1         
        p[0]=pivotRow               

                         7   7 10    7      7 10
1st column/val_check     2/7 2  8    0.2857 2 8
val_check = 7            6/7 8  1    0.8571 8 1

                                                 7   7  10
  2 8  -=  2/7  @ 7  10  -= 2 20/7   = 0  36/7   2/7 0  36/7
  8 1      6/7              6 60/7     2 -53/7   6/7 2 -53/7

2nd round :                           P = 1 2 2 
 7   7  10    max(0,2)=2               7 7 10
 2/7 0  36/7  pivotRow = 2            6/7 2 -53/7   val check = 2   0/2
 6/7 2 -53/7  rowswap(row2,row1)      2/7 0  36/7              no change
val_check = 0   we cant /val_check

36/7 -= 0@-53/7  no change  

                                      1           2          2 no change
final is just pivotrow    P = 1 2 2   v           v          v
7   7    10               this means  0 1 2  -> 1 0 2 -> 1 2 0   
6/7 2 -53/7                           
2/7 0  36/7 

so pivoting is 1 2 0   
you just L@U  and then change with row 1 2 0 swap  
```


In [ ]:
python run.py  --opt -1 --size 3 --pivoting 0  --check_equal 1 \
  --verbose false --mc_iter 100
python run.py  --opt -1 --size 3 --pivoting 1  --check_equal 1 \
  --verbose false --mc_iter 100

### without pivoting just works fine
### with pivoting
```
mat= 935 485 750 
    i = 0                    swap(0,0)     upper      lower[:,0] /upper[0]=/9
9 3 5 max_val=max(947)=9                   9 3 5        1  0 0 
4 8 5 pivot = 0            doesnt change   0 0 0       4/9 0 0
7 5 0 P=012,                               0 0 0       7/9 0 0  

    i = 1  
max_val=max(85)=8                 upper
pivot = 1                         9   3    5
swap(1,1) no change               0 20/3  25/9
upper update row[1,1:]            0   0    0 

k=1: 8 - 4/9*3 = 20/3 =6.667 
k=2: 5 - 4/9*5 = 25/9 = 2.777            

lower 
1   0   0   k = 1 (set 1)   k= 2   sum += lower[2][0]*upper[0][1]=7/9*3
4/9 1   0                          lower[2][1]=(mat[2][1]-sum)/upper[1][1]
7/9 0.4 0                            = (5-21/9)/(20/3) = 0.4

    i = 2  
max_val = 0
```

#### how doens cuda ver deal with this ??  

In [ ]:
python run.py  --opt 1 --size 3 --pivoting 1  --check_equal 1   --verbose false --mc_iter 1
00

```
mat= 935 485 750 
    i = 0                    swap(0,0)     upper      lower[:,0] /upper[0]=/9
9 3 5 max_val=max(947)=9                   9 3 5        1  0 0 
4 8 5 pivot = 0            doesnt change   0 0 0       4/9 0 0
7 5 0 P=012,                               0 0 0       7/9 0 0  
```

todo this will errr  

In [ ]:
python run.py  --opt -2 --size 3 --pivoting 1 \
   --check_equal 1   --verbose true --mc_iter 1000